In [1]:
import pandas
import numpy
from sklearn.preprocessing import LabelEncoder
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler

In [2]:
dataset = pandas.read_csv('../dataset/alb-rainfall-adm2-full.csv')

C:\Users\Admin\AppData\Local\Temp\ipykernel_12988\1204217597.py:1: DtypeWarning: Columns (1,3,4,5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pandas.read_csv('../dataset/alb-rainfall-adm2-full.csv')


In [3]:
categorical_columns = [column for column in dataset.columns if dataset[column].dtype=='object']
print("Categorical columns: ",categorical_columns)

numerical_columns = [column for column in dataset.columns if dataset[column].dtype!='object']
print("Numerical columns: ",numerical_columns)


Categorical columns:  ['date', 'adm2_id', 'ADM2_PCODE', 'n_pixels', 'rfh', 'rfh_avg', 'r1h', 'r1h_avg', 'r3h', 'r3h_avg', 'rfq', 'r1q', 'r3q', 'version']
Numerical columns:  []


In [4]:
numerical_columns = ["n_pixels", "rfh", "rfh_avg", "r1h", "r1h_avg", "r3h", "r3h_avg", "rfq", "r1q", "r3q"]
for column in numerical_columns:
    dataset[column] = pandas.to_numeric(dataset[column],errors='coerce')

In [5]:
print("Numerical columns: ",numerical_columns)

for column in numerical_columns:
    print(f"Unique values in {column}: ", dataset[column].unique())

Numerical columns:  ['n_pixels', 'rfh', 'rfh_avg', 'r1h', 'r1h_avg', 'r3h', 'r3h_avg', 'rfq', 'r1q', 'r3q']
Unique values in n_pixels:  [nan  2.  1.  3.  4.  6. 14.  5. 10.  9.  8.  7. 16. 11. 13.]
Unique values in rfh:  [  nan  20.   73.5 ... 267.2 167.8 172.4]
Unique values in rfh_avg:  [    nan 38.9167 31.6167 ... 57.6917 48.1917 41.5083]
Unique values in r1h:  [  nan  99.5 118.5 ... 372.4 427.2 388.4]
Unique values in r1h_avg:  [     nan 117.     121.7333 ... 137.5083 116.4833 109.425 ]
Unique values in r3h:  [  nan 391.5 380.  ... 952.8 937.6 745.4]
Unique values in r3h_avg:  [     nan 358.5667 354.2667 ... 375.4    339.1917 327.2   ]
Unique values in rfq:  [     nan  56.926  214.3832 ... 143.547   79.0716  26.2977]
Unique values in r1q:  [     nan  86.2205  97.5455 ... 116.6036 107.6119  74.9111]
Unique values in r3q:  [     nan 108.9355 107.0644 ...  78.2117  70.5813  81.2792]


In [6]:
dataset.replace({'?': None, 'missing': None, 'N/A': None}, inplace=True)

In [7]:
numerical_columns = [col for col in dataset.columns if dataset[col].dtype in ['int64', 'float64']]
print("Numerical columns: ", numerical_columns)
print(dataset.isnull().sum())

Numerical columns:  ['n_pixels', 'rfh', 'rfh_avg', 'r1h', 'r1h_avg', 'r3h', 'r3h_avg', 'rfq', 'r1q', 'r3q']
date             0
adm2_id          0
ADM2_PCODE       0
n_pixels         1
rfh              1
rfh_avg          1
r1h            703
r1h_avg        703
r3h           2809
r3h_avg       2809
rfq              1
r1q            703
r3q           2809
version          0
dtype: int64


In [8]:
dataset[['n_pixels', 'rfh', 'rfh_avg', 'rfq']] = dataset[['n_pixels', 'rfh', 'rfh_avg', 'rfq']].fillna(dataset[['n_pixels', 'rfh', 'rfh_avg', 'rfq']].median())
dataset[['r1h', 'r1h_avg', 'r1q']] = dataset[['r1h', 'r1h_avg', 'r1q']].fillna(dataset[['r1h', 'r1h_avg', 'r1q']].median())
dataset[['r3h', 'r3h_avg', 'r3q']] = dataset[['r3h', 'r3h_avg', 'r3q']].fillna(dataset[['r3h', 'r3h_avg', 'r3q']].median())

In [9]:
dataset['date'] = pandas.to_datetime(dataset['date'], errors='coerce', format='%Y-%m-%d')
print(dataset['date'].head())

0          NaT
1   1981-01-01
2   1981-01-11
3   1981-01-21
4   1981-02-01
Name: date, dtype: datetime64[ns]


In [10]:
print(dataset[dataset['date'].isna()])
dataset.drop(index=0, inplace=True)
print(dataset.isnull().sum())

  date   adm2_id  ADM2_PCODE  n_pixels   rfh  rfh_avg      r1h  r1h_avg  \
0  NaT  #adm2+id  #adm2+code       3.0  23.0  32.8333  84.3333    99.85   

     r3h  r3h_avg      rfq      r1q      r3q  version  
0  273.0  299.725  81.7095  91.8176  95.3794  #status  
date          0
adm2_id       0
ADM2_PCODE    0
n_pixels      0
rfh           0
rfh_avg       0
r1h           0
r1h_avg       0
r3h           0
r3h_avg       0
rfq           0
r1q           0
r3q           0
version       0
dtype: int64


In [11]:
dataset = pandas.get_dummies(dataset, columns=['version'], drop_first=True)
encoder = LabelEncoder()
dataset['ADM2_PCODE'] = encoder.fit_transform(dataset['ADM2_PCODE'])

In [12]:
z_score = dataset[numerical_columns].apply(zscore)
outliers = (z_score.abs()>3)
print("Number of outliers per column (Z-Score):\n", outliers.sum())

Number of outliers per column (Z-Score):
 n_pixels     6356
rfh         10794
rfh_avg      6032
r1h          8561
r1h_avg      5896
r3h          6642
r3h_avg      4794
rfq          8076
r1q          4598
r3q          3268
dtype: int64


In [13]:
columns_to_transform = ["rfh", "rfh_avg", "r1h", "r1h_avg", "r3h", "r3h_avg", "rfq", "r1q", "r3q"]
dataset[columns_to_transform] = dataset[columns_to_transform].apply(lambda x: numpy.log1p(x))

In [14]:
scaler = StandardScaler()
dataset[numerical_columns] = scaler.fit_transform(dataset[numerical_columns])

In [15]:
dataset['year_month'] = dataset['date'].dt.to_period('M')

In [16]:
aggregated_data = dataset.groupby(['adm2_id', 'ADM2_PCODE', 'year_month']).agg({
    'n_pixels': 'mean',
    'rfh': 'mean',
    'rfh_avg': 'mean',
    'r1h': 'mean',
    'r1h_avg': 'mean',
    'r3h': 'mean',
    'r3h_avg': 'mean',
    'rfq': 'mean',
    'r1q': 'mean',
    'r3q': 'mean'
}).reset_index()